上次说了很多Linux下进程相关知识，这边不再复述

如果遇到听不懂的可以看上一次的文章：<a href="https://www.cnblogs.com/dotnetcrazy/p/9363810.html" target="_blank">https://www.cnblogs.com/dotnetcrazy/p/9363810.html</a>

下面来说说Python的并发编程，如有错误欢迎提出～

## 1.进程篇

官方文档：<a href="https://docs.python.org/3/library/multiprocessing.html" target="_blank">https://docs.python.org/3/library/multiprocessing.html</a>

### 1.1.进程（Process）

Python的进程创建非常方便，看个案例：(这种方法通用，fork只适用于Linux系)
```py
import os
# 注意一下，导入的是Process不是process（Class是大写开头）
from multiprocessing import Process

def test(name):
    print("[子进程-%s]PID：%d，PPID：%d" % (name, os.getpid(), os.getppid()))

def main():
    print("[父进程]PID：%d，PPID：%d" % (os.getpid(), os.getppid()))
    p = Process(target=test, args=("萌萌哒", )) # 单个元素的元组表达别忘了(x,)
    p.start()
    p.join()  # 父进程回收子进程资源（内部调用了wait系列方法）

if __name__ == '__main__':
    main()
```
运行结果：
```
[父进程]PID：25729，PPID：23434
[子进程-萌萌哒]PID：25730，PPID：25729
```

创建子进程时，传入一个执行函数和参数，用start()方法来启动进程即可

`join()`方法是父进程回收子进程的封装（主要是回收<a href="https://www.cnblogs.com/dotnetcrazy/p/9363810.html#2.2.僵尸进程和孤儿进程" target="_blank">僵尸子进程(点我)</a>）

其他参数可以参考源码 or 文档，贴一下源码的`init`方法：

`def __init__(self,group=None,target=None,name=None,args=(),kwargs={},*,daemon=None)`

扩展：`name：为当前进程实例的别名`

1. p.is_alive() 判断进程实例p是否还在执行
2. p.terminate() 终止进程实例p

---

### 1.1.源码拓展

现在说说里面的一些门道（只像用的可以忽略）

新版本的封装可能多层，这时候可以看看Python3.3.X系列（这个算是Python3早期版本了，很多代码都暴露出来，比较明了直观）

multiprocessing.process.py
```py
# 3.4.x开始，Process有了一个BaseProcess
# https://github.com/python/cpython/blob/3.7/Lib/multiprocessing/process.py
def join(self, timeout=None):
    '''一直等到子进程over'''
    self._check_closed()
    # 断言（False就触发异常，提示就是后面的内容
    # 开发中用的比较多，部署的时候可以python3 -O xxx 去除所以断言
    assert self._parent_pid == os.getpid(), "只能 join 一个子进程"
    assert self._popen is not None, "只能加入一个已启动的进程"
    res = self._popen.wait(timeout) # 本质就是用了我们之前讲的wait系列
    if res is not None:
        _children.discard(self) # 销毁子进程
```
multiprocessing.popen_fork.py
```py
# 3.4.x开始，在popen_fork文件中（以前是multiprocessing.forking.py）
# https://github.com/python/cpython/blob/3.7/Lib/multiprocessing/popen_fork.py
def wait(self, timeout=None):
    if self.returncode is None:
        # 设置超时的一系列处理
        if timeout is not None:
            from multiprocessing.connection import wait
            if not wait([self.sentinel], timeout):
                return None
        # 核心操作
        return self.poll(os.WNOHANG if timeout == 0.0 else 0)
    return self.returncode

# 回顾一下上次说的：os.WNOHANG - 如果没有子进程退出，则不阻塞waitpid()调用
def poll(self, flag=os.WNOHANG):
    if self.returncode is None:
        try:
            # 他的内部调用了waitpid
            pid, sts = os.waitpid(self.pid, flag)
        except OSError as e:
            # 子进程尚未创建
            # e.errno == errno.ECHILD == 10
            return None
        if pid == self.pid:
            if os.WIFSIGNALED(sts):
                self.returncode = -os.WTERMSIG(sts)
            else:
                assert os.WIFEXITED(sts), "Status is {:n}".format(sts)
                self.returncode = os.WEXITSTATUS(sts)
    return self.returncode
```

关于断言的简单说明：（别泛滥）

如果条件为真，它什么都不做，反之它触发一个带可选错误信息的AssertionError

```py
def test(a, b):
    assert b != 0, "哥哥，分母不能为0啊"
    return a / b

def main():
    test(1, 0)

if __name__ == '__main__':
    main()
```
结果：
```
Traceback (most recent call last):
  File "0.assert.py", line 11, in <module>
    main()
  File "0.assert.py", line 7, in main
    test(1, 0)
  File "0.assert.py", line 2, in test
    assert b != 0, "哥哥，分母不能为0啊"
AssertionError: 哥哥，分母不能为0啊
```
运行的时候可以指定`-O参数`来忽略所以`assert`，eg：

`python3 -O 0.assert.py `
```
Traceback (most recent call last):
  File "0.assert.py", line 11, in <module>
    main()
  File "0.assert.py", line 7, in main
    test(1, 0)
  File "0.assert.py", line 3, in test
    return a / b
ZeroDivisionError: division by zero
```

---

扩展：

<a href="https://docs.python.org/3/library/unittest.html" target="_blank">https://docs.python.org/3/library/unittest.html</a>

<a href="https://www.cnblogs.com/shangren/p/8038935.html" target="_blank">https://www.cnblogs.com/shangren/p/8038935.html</a>

### 1.2.进程池

多个进程就不需要自己手动去管理了，有Pool来帮你完成，先看个案例：